# Overview

This notebook pulls down data off the ROS 2 build farm and does some preprocessing work to make it much more easy to work with. The build farm runs a variety of tests and for each test generates a single csv file that are zipped together. This notebook takes all of those notebooks, which are really rough, and merges them together into a handful of larger CSV files that are a lot easier to work with using pandas. 

There are three types of test data that come off the build farm:

1. `overhead_node` These files evaluate a single spinning ROS node in terms of cpu / memory consumption and a few other metrics. 
2. `overhead_tests` These tests examine interop between different RMW vendors where for two different vendors one acts as a publishing node and the other acts as a subscriber node. These tests profile the performance for this network confiration
3. `two_process_perf` These tests create a publisher and subscriber that use the same RMW vendor. The nodes send messages from publisher to subscriber and the whole assembly is instrumented to collect system performance and networking performance data. 

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import glob as glob
# Blessed build for evaluation is
# http://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/97/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
# The next block will pull down the zip file and extract it to the correct location 

In [4]:
! wget http://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/97/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
! mkdir ./data/build_farm/raw/
! mv buildfarm_perf_tests.zip ./data/build_farm/raw/
! unzip ./data/build_farm/raw/buildfarm_perf_tests.zip -d ./data/build_farm/raw/ 

--2020-10-22 17:06:32--  http://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/97/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
Resolving build.ros2.org (build.ros2.org)... 13.52.151.147
Connecting to build.ros2.org (build.ros2.org)|13.52.151.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘buildfarm_perf_tests.zip’

buildfarm_perf_test     [ <=>                ]  83.19K  --.-KB/s    in 0.1s    

2020-10-22 17:06:32 (869 KB/s) - ‘buildfarm_perf_tests.zip’ saved [85189]

Archive:  ./data/build_farm/raw/buildfarm_perf_tests.zip
  inflating: ./data/build_farm/raw/overhead_node_test_results_rmw_connext_cpp_async.csv  
  inflating: ./data/build_farm/raw/overhead_node_test_results_rmw_cyclonedds_cpp_sync.csv  
  inflating: ./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_cpp_async.csv  
  inflating: ./data/build_farm/raw/overhead_node_test_results_rmw_fastrt

In [5]:
# First let's try to figure out blocks of data
# I.e. what are the "sets" of files we can process.
out = glob.glob("./data/build_farm/raw/*.csv")

print("Total Files: {0}".format(len(out)))    
perf_files = [f for f in out if "performance" in f]
print("Performance Files: {0}".format(len(perf_files)))
overhead_files = [f for f in out if "overhead" in f]
print("Overhead Files: {0}".format(len(overhead_files)))
two_files = [f for f in out if "two_process" in f]
print("Two Process Files: {0}".format(len(two_files)))
sync_files = [f for f in out if "_sync" in f]
print("Sync Files: {0}".format(len(sync_files)))
async_files = [f for f in out if "async" in f]
print("Async Files: {0}".format(len(async_files)))
pub_files = [f for f in out if "_pub" in f]
print("pub Files: {0}".format(len(pub_files)))
sub_files = [f for f in out if "_sub" in f]
print("sub Files: {0}".format(len(sub_files)))
node_files = [f for f in out if "node" in f]
print("node Files: {0}".format(len(node_files)))


Total Files: 230
Performance Files: 176
Overhead Files: 54
Two Process Files: 88
Sync Files: 115
Async Files: 115
pub Files: 24
sub Files: 24
node Files: 6


In [6]:
perf_cols = ['mean virtual memory (Mb)',
             'median virtual memory (Mb)',
             'virtual memory (Mb)',
             'mean cpu_usage (%)',
             'median cpu_usage (%)',
             'cpu_usage (%)',
             'mean physical memory (Mb)',
             'median physical memory (Mb)',
             'physical memory (Mb)',
             'mean resident anonymous memory (Mb)',
             'median resident anonymous memory (Mb)',
             'resident anonymous memory (Mb)']

In [8]:
# Take all of the "overhead" files and try to merge them into a single table. 
for p in node_files:
    print(p)

df = pd.read_csv(node_files[0])
df.columns = perf_cols
for p in node_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_cols
    df = df.append(temp)
# parse the filenames and add that data. 
df["config"] = ["_".join(n.strip('./data/build_farm/raw/overhead_node_test_results_rmw_').strip('.csv').split('_')[1:]) for n in node_files]
df["vendor"] = [n.strip('./data/build_farm/raw/overhead_node_test_results_rmw_').split('_')[0] for n in node_files]
df = df[df.columns[::-1]]
df["file_name"] = node_files
df.to_csv("./data/build_farm/node_perf.csv")
print(len(df))
print(df["file_name"])

./data/build_farm/raw/overhead_node_test_results_rmw_connext_cpp_async.csv
./data/build_farm/raw/overhead_node_test_results_rmw_cyclonedds_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_dynamic_cpp_async.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_dynamic_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_cpp_async.csv
6
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
Name: file_name, dtype: object


In [9]:
def fname_to_data(fname, head="./data/build_farm/raw/overhead_test_results_rmw_",tail="_ROS2_pub.csv"):
    """
    Munge a file name into metadata. Pull out the first and seond RMW 
    along with the "flavor" information
    """
    fname = fname.replace(head,"").replace(tail,"")
    parts = fname.split("_rmw_")
    first = parts[0].split("_")
    second = parts[1].split("_")
    # format is rmw _ <name> _ <config> _ rwm _ <name2> _ <config2>
    ret_val = {}
    ret_val["first_rmw"] = first[0]
    ret_val["second_rmw"] = second[0]
    ret_val["first_flavor"] = " ".join(first[1:])
    ret_val["second_flavor"] = " ".join(second[1:])
    return(ret_val)

fname_to_data("./data/build_farm/raw/overhead_test_results_rmw_fastrtps_cpp_async_rmw_connext_cpp_ROS2_pub.csv")

{'first_flavor': 'cpp async',
 'first_rmw': 'fastrtps',
 'second_flavor': 'cpp',
 'second_rmw': 'connext'}

In [10]:
pub_sub_cols = ['mean virtual memory (Mb)',
                'median virtual memory (Mb)',
                'virtual memory (Mb)',
                'mean cpu_usage (%)',
                'median cpu_usage (%)',
                'cpu_usage (%)',
                'mean physical memory (Mb)',
                'median physical memory (Mb)',
                'physical memory (Mb)',
                'mean resident anonymous memory (Mb)',
                'median resident anonymous memory (Mb)',
                'resident anonymous memory (Mb)',
                'mean latency_mean (ms)',
                'median latency_mean (ms)',
                'Top 5% latency (ms)',
                'max ru_maxrss',
                'mean received',
                'mean sent',
                'sum lost',
                'mean system_cpu_usage (%)',
                'mean system virtual memory (Mb)']

In [11]:
# Pull out data for the pub files and repeat for sub files. 
pub_df = pd.read_csv(pub_files[0])
print(pub_files[0])

print("DF Cols {0} vs known cols {1}".format(len(pub_df.columns),len(pub_sub_cols)))    
# squish all the files into one table
pub_df.columns = pub_sub_cols
for p in pub_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = pub_sub_cols
    pub_df = pub_df.append(temp)
# parse the file names into data and add them back to table. 
flavors = [fname_to_data(flavor) for flavor in pub_files]
pub_df["from_rmw"]= [flavor["first_rmw"] for flavor in flavors]
pub_df["from_rmw_flavor"]= [flavor["first_flavor"] for flavor in flavors]
pub_df["to_rmw"]= [flavor["second_rmw"] for flavor in flavors]
pub_df["to_rmw_flavor"]= [flavor["second_flavor"] for flavor in flavors]
pub_df["file_name"] = pub_files
pub_df = pub_df[pub_df.columns[::-1]]
pub_df.to_csv("./data/build_farm/pub_perf.csv")
pub_df.head()

./data/build_farm/raw/overhead_test_results_rmw_connext_cpp_async_rmw_fastrtps_cpp_ROS2_pub.csv
DF Cols 21 vs known cols 21


,file_name,to_rmw_flavor,to_rmw,from_rmw_flavor,from_rmw,mean system virtual memory (Mb),mean system_cpu_usage (%),sum lost,mean sent,mean received,...,mean resident anonymous memory (Mb),physical memory (Mb),median physical memory (Mb),mean physical memory (Mb),cpu_usage (%),median cpu_usage (%),mean cpu_usage (%),virtual memory (Mb),median virtual memory (Mb),mean virtual memory (Mb)
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,cpp async,connext,1560.774324,25.523836,0.0,4.480000,4.48,...,16.288288,68.88512,66.4141,65.153218,2.561014,2.40220,2.333279,604.922,604.922,581.998058
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,dynamic cpp sync,fastrtps,1505.056774,28.918977,0.0,4.480000,4.48,...,9.462225,38.97270,38.9727,37.848959,2.596975,2.37607,2.280575,146.876,146.876,142.411975
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,dynamic cpp async,fastrtps,1506.325484,29.220684,0.0,4.480000,4.48,...,9.443550,38.94530,38.9453,37.774181,2.231375,2.12424,1.959387,162.865,162.865,157.885200
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,connext,cpp sync,cyclonedds,1551.537742,30.820745,0.0,4.458333,4.50,...,6.230845,25.66800,25.6680,24.923417,3.733285,3.36405,3.194584,166.826,166.826,161.718017
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,cpp sync,fastrtps,1552.795806,29.120223,0.0,4.480000,4.52,...,9.489924,39.16410,39.1641,37.959714,2.321840,2.27273,2.132442,146.850,146.850,142.337544


In [12]:
# Now repeat for subscribersub_perf.head()
sub_df = pd.read_csv(sub_files[0])
print(sub_files[0])

print("DF Cols {0} vs known cols {1}".format(len(sub_df.columns),len(pub_sub_cols)))    

sub_df.columns = pub_sub_cols
for p in sub_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = pub_sub_cols
    sub_df = sub_df.append(temp)
    
flavors = [fname_to_data(flavor,tail="_ROS2_sub.csv") for flavor in sub_files]
sub_df["from_rmw"]= [flavor["first_rmw"] for flavor in flavors]
sub_df["from_rmw_flavor"]= [flavor["first_flavor"] for flavor in flavors]
sub_df["to_rmw"]= [flavor["second_rmw"] for flavor in flavors]
sub_df["to_rmw_flavor"]= [flavor["second_flavor"] for flavor in flavors]
sub_df["file_name"] = sub_files
sub_df = sub_df[sub_df.columns[::-1]]
sub_df.to_csv("./data/build_farm/sub_perf.csv")
sub_df.head()

./data/build_farm/raw/overhead_test_results_rmw_connext_cpp_async_rmw_fastrtps_cpp_ROS2_sub.csv
DF Cols 21 vs known cols 21


,file_name,to_rmw_flavor,to_rmw,from_rmw_flavor,from_rmw,mean system virtual memory (Mb),mean system_cpu_usage (%),sum lost,mean sent,mean received,...,mean resident anonymous memory (Mb),physical memory (Mb),median physical memory (Mb),mean physical memory (Mb),cpu_usage (%),median cpu_usage (%),mean cpu_usage (%),virtual memory (Mb),median virtual memory (Mb),mean virtual memory (Mb)
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,cpp async,connext,1564.635000,29.356643,0.0,0.0,4.52,...,9.549807,39.2070,39.2070,38.199189,27.073275,26.85455,25.668276,162.852,162.852,157.706543
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,cyclonedds,cpp sync,cyclonedds,1531.771935,29.680961,0.0,0.0,4.48,...,6.576457,27.1953,27.1953,26.314128,27.166600,26.85610,25.646199,150.827,150.576,146.016815
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,cpp sync,fastrtps,1552.887742,29.094513,0.0,0.0,4.48,...,9.454635,38.9297,38.9297,37.818561,27.187400,27.06020,25.778357,146.827,146.827,142.364650
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,dynamic cpp sync,fastrtps,1505.345667,29.332597,0.0,0.0,4.50,...,9.670703,39.6133,39.6133,38.682830,27.046910,26.62155,24.952088,146.877,146.877,142.264043
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,cpp sync,fastrtps,1553.772333,29.630393,0.0,0.0,4.52,...,9.560611,39.3516,39.3516,38.242484,26.777540,26.27845,24.747906,146.879,146.879,142.266171


In [13]:
# now aggregate the performance results, there are two types two process and and "results"
two_process_perf = [p for p in perf_files if "two_process" in p]
result_perf_file = [p for p in perf_files if "two_process" not in p]
print("{0} two process files and {1} results files. {2} total files.".format(len(two_process_perf),len(result_perf_file),len(perf_files)))

# From: https://github.com/ahcorde/buildfarm_perf_tests/blob/master/test/test_performance.py.in#L48
perf_col_names = [
    'mean latency_mean (ms)',
    'median latency_mean (ms)',
    '95th Percentile Latency',
    'max ru_maxrss',
    'mean received',
    'mean sent',
    'sum lost',
    'mean cpu_usage (%)',
    '95th Percentile CPU',
    'median cpu_usage (%)',
    'mean data_received (Mb)',
    'median data_received (Mb)',
    '95th Percentile Data Received (Mb)']


88 two process files and 88 results files. 176 total files.


In [14]:
def fname_to_rmw_and_data(fname):
    """
    Parse and return file names of the format
    performnace_test_resuts_<optional rmw>_<rmw_name>_<rmw_flavor>_<datatype>.csv
    E.g. 
    ./data/performance_test_results_rmw_fastrtps_dynamic_cpp_async_Array32k.csv
    ./data/performance_test_results_FastRTPS_sync_Array2m.csv
    ./data/performance_test_results_CycloneDDS_sync_Array1k.csv
    """
    fname = fname.replace("./data/build_farm/raw/performance_test_two_process_results_rmw_","")
    fname = fname.replace("./data/build_farm/raw/performance_test_two_process_results_","")
    fname = fname.replace("./data/build_farm/raw/performance_test_results_","")
    
    fname = fname.replace(".csv","")
    parts = fname.split("_");
    ret_val = {}
    ret_val["type"] = parts[-1] # last entry is type, easy
    if(parts[0] == "rmw"):
        parts = parts[1:] # drop the first value if it is RMW
    ret_val["vendor"] = parts[0].lower() # both upper and lower is present
    ret_val["flavor"] = "_".join(parts[1:-1])
    return ret_val 

In [17]:
perf_df = pd.read_csv(result_perf_file[0])
print(result_perf_file[0])

print("DF Cols {0} vs known cols {1}".format(len(perf_df.columns),len(perf_col_names)))

perf_df.columns = perf_col_names

# smush main csv files together
for p in result_perf_file[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_col_names
    perf_df = perf_df.append(temp)
# parse file names 
fname_data = [fname_to_rmw_and_data(p) for p in result_perf_file]
perf_df["vendor"] = [p["vendor"] for p in fname_data]
perf_df["flavor"] = [p["flavor"] for p in fname_data]
perf_df["data_type"] = [p["type"] for p in fname_data]
perf_df["file_name"] = result_perf_file
perf_df = perf_df[perf_df.columns[::-1]]
perf_df.to_csv("./data/build_farm/perf_network_results.csv")
perf_df.head()


./data/build_farm/raw/performance_test_results_rmw_cyclonedds_cpp_sync_Array4m.csv
DF Cols 13 vs known cols 13


,file_name,data_type,flavor,vendor,95th Percentile Data Received (Mb),median data_received (Mb),mean data_received (Mb),median cpu_usage (%),95th Percentile CPU,mean cpu_usage (%),sum lost,mean sent,mean received,max ru_maxrss,95th Percentile Latency,median latency_mean (ms),mean latency_mean (ms)
0,./data/build_farm/raw/performance_test_results...,Array4m,cpp_sync,cyclonedds,4488.681515,4000.105199,3999.920211,29.970,44.655,32.101852,0.0,999.555556,999.555556,87492.0,4.862500,1.17500,1.668819
0,./data/build_farm/raw/performance_test_results...,Array4m,sync,fastrtps,1085.367997,1011.817813,1013.845989,26.490,27.387,26.480370,20163.0,999.185185,252.555556,130016.0,12.894000,11.70000,11.674444
0,./data/build_farm/raw/performance_test_results...,Array2m,cpp_sync,fastrtps,2000.219628,2000.004910,2000.005110,15.980,16.237,15.935556,0.0,999.407407,999.407407,91480.0,0.626490,0.61190,0.612081
0,./data/build_farm/raw/performance_test_results...,Array60k,cpp_async,connext,58.612989,58.609075,58.608791,6.494,6.744,6.604556,0.0,999.555556,999.555556,90084.0,0.166960,0.15890,0.159700
0,./data/build_farm/raw/performance_test_results...,Array60k,async,fastrtps,58.657322,58.599196,58.607739,1.250,1.500,1.286993,0.0,999.148148,999.148148,91316.0,0.042939,0.03606,0.035301


In [18]:
twop_df = pd.read_csv(two_process_perf[0])
print(two_process_perf[0])

print("DF Cols {0} vs known cols {1}".format(len(twop_df.columns),len(perf_col_names)))

twop_df.columns = perf_col_names

# smush main csv files together
for p in two_process_perf[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_col_names
    twop_df = twop_df.append(temp)
# parse file names 
fname_data = [fname_to_rmw_and_data(p) for p in two_process_perf]
twop_df["vendor"] = [p["vendor"] for p in fname_data]
twop_df["flavor"] = [p["flavor"] for p in fname_data]
twop_df["data_type"] = [p["type"] for p in fname_data]
twop_df["file_name"] = two_process_perf
twop_df = twop_df[twop_df.columns[::-1]]
twop_df.to_csv("two_process_perf_network_results.csv")
twop_df.head()


./data/build_farm/raw/performance_test_two_process_results_rmw_connext_cpp_async_Array1k.csv
DF Cols 13 vs known cols 13


,file_name,data_type,flavor,vendor,95th Percentile Data Received (Mb),median data_received (Mb),mean data_received (Mb),median cpu_usage (%),95th Percentile CPU,mean cpu_usage (%),sum lost,mean sent,mean received,max ru_maxrss,95th Percentile Latency,median latency_mean (ms),mean latency_mean (ms)
0,./data/build_farm/raw/performance_test_two_pro...,Array1k,cpp_async,connext,0.991859,0.991829,0.990938,2.4980,2.74700,2.642185,0.0,0.0,998.740741,90008.0,0.085261,0.07970,0.080251
0,./data/build_farm/raw/performance_test_two_pro...,Array8m,async,fastrtps,87.986031,47.989203,45.917634,0.7499,1.75000,0.740700,18197.0,0.0,4.962963,91564.0,26.627000,22.52000,17.980000
0,./data/build_farm/raw/performance_test_two_pro...,Array4k,dynamic_cpp_sync,fastrtps,3.921604,3.921521,3.917149,0.9990,1.17403,0.973841,0.0,0.0,998.481481,91712.0,0.035761,0.03452,0.034385
0,./data/build_farm/raw/performance_test_two_pro...,Array16k,cpp_async,fastrtps,15.641267,15.640259,15.625202,0.9990,1.24900,1.093152,0.0,0.0,998.592593,91608.0,0.063282,0.05588,0.056342
0,./data/build_farm/raw/performance_test_two_pro...,Array8m,cpp_sync,cyclonedds,740.264812,607.905283,591.667856,26.8800,28.68900,26.136481,2745.0,0.0,73.444444,95248.0,11.527000,10.80000,10.881111


In [21]:
total = len(pub_files)+len(sub_files)+len(node_files)+len(two_process_perf)+len(result_perf_file)
print("processed {0} of {1}".format(total,len(glob.glob("./data/build_farm/raw/*.csv"))))

processed 230 of 230
